In [1]:
#Install these libraries (just remove the hashtag) if you don't have them installed already. You can delete this part afterwards.
#!pip install biopython
#!pip install pandas
#!pip install requests

In [20]:
import pandas as pd
import requests
import time
import re
from io import StringIO
from Bio.Blast import NCBIWWW, NCBIXML
import Bio

In [3]:
#Loads CSV. Replace the file name in quotations, make sure the script and the file are in the same folder.
df = pd.read_csv("methyorg_complete.csv") #*******Put your file name in, it has to be in .csv format.

In [4]:
##I think I've been going about this code backwards. I think instead what I'll do is search for the EC number in KEGG -> Use the EC number to pBLAST -> Grab the GPR from there.
##May need to expand the acceptable amounts.

#==========================
#==========================
# KEGG -> EC Converter             
#==========================
#==========================

def KEGG_ID_to_EC_kegg(kegg_id):
    url = f"http://rest.kegg.jp/get/rn:{kegg_id}"
    response = requests.get(url)
    if response.status_code != 200:
        return ""
    ec_numbers = []
    for line in response.text.split('\n'):
        if line.startswith("ENZYME"):
            ec_numbers = line.split()[1:]  # all EC numbers after 'ENZYME'
            break
    return "; ".join(ec_numbers) if ec_numbers else ""

#Loop through and build list of EC numbers
ec_list = []
for kegg_id in df['KEGG ID']:
    if pd.isna(kegg_id) or kegg_id.strip() == "":
        ec_list.append("")
    else:
        ecs = KEGG_ID_to_EC_kegg(kegg_id.strip())
        ec_list.append(ecs)
        print(f"{kegg_id} → ECs: {ecs}")
        time.sleep(0.5)

#Adds to current df
df['KEGG_EC'] = ec_list
print("✅ KEGG EC column added and saved.")

R07394; R07395 → ECs: 3.1.3.77; 3.1.3.87
R04544 → ECs: 2.3.1.85; 2.3.1.86; 2.3.1.-; 4.2.1.59
R00006; R00226 → ECs: 2.2.1.6
R00259 → ECs: 2.3.1.1
R07759 → ECs: 1.1.1.330
R04212 → ECs: 6.3.5.6
R00400 → ECs: 2.6.1.12
R04957 → ECs: 2.3.1.41; 2.3.1.85; 2.3.1.86; 2.3.1.179
R01274 → ECs: 3.1.2.2; 3.1.2.22; 3.1.2.-
R07268 → ECs: 2.5.1.17
R04533 → ECs: 1.1.1.100; 1.1.1.-; 2.3.1.85; 2.3.1.86
R02568 → ECs: 4.1.2.13
R03905 → ECs: 6.3.5.7
R00588 → ECs: 2.6.1.45
R00465 → ECs: 1.1.1.79
R00475 → ECs: 1.1.3.15
R00650 → ECs: 2.1.1.10
R04640 → ECs: 5.3.1.16
R03503 → ECs: 2.7.6.3
R05338 → ECs: 4.1.2.43
R04567 → ECs: 2.3.1.129
R00946 → ECs: 2.1.1.13
R00294 → ECs: 1.7.2.5
R00291 → ECs: 5.1.3.2
R02719 → ECs: 3.1.3.5
R00717 → ECs: 1.1.1.26; 1.1.1.29
R00662 → ECs: 3.6.1.9; 3.6.1.-
R02017 → ECs: 1.17.4.1
R02019 → ECs: 1.17.4.1
R00774 → ECs: 6.3.4.6
R07302 → ECs: 6.3.1.10
R06529 → ECs: 6.3.1.10
R05185 → ECs: 1.18.6.1
R01071 → ECs: 2.4.2.17
R00965 → ECs: 4.1.1.23
R03182 → ECs: 6.3.3.3
R01142 → ECs: 1.14.13.25
R04

In [25]:
##Next, we'll convert these into their protein sequences on Uniprot based on their updated EC numbers

def fetch_uniprot_fasta_by_ec(ec_number):
    base_url = "https://rest.uniprot.org/uniprotkb/search"
    query = f"ec:{ec_number}"

    params = {
        "query": query,
        "format": "fasta",
        "size": 1
    }

    try:
        response = requests.get(base_url, params=params)
        if response.status_code == 200 and response.text.strip():
            return response.text
        else:
            return "No sequence found"
    except Exception as e:
        print(f"Error fetching for EC {ec_number}: {e}")
        return "No sequence found"

#Loop through KEGG_EC column, fetch UniProt sequence
uniprot_seqs = []
for ecs in df['KEGG_EC']:
    if pd.isna(ecs) or ecs.strip() == "":
        uniprot_seqs.append("No EC number")
        continue

    ec_list = [ec.strip() for ec in ecs.split('; ') if ec.strip()]
    seq_found = "No sequence found"

    for ec in ec_list:
        result = fetch_uniprot_fasta_by_ec(ec)
        if "No sequence" not in result and "not found" not in result:
            #Remove header lines and join sequence
            sequence_only = "".join([
                line.strip() for line in result.split('\n') if not line.startswith(">")
            ])
            seq_found = sequence_only
            break

    uniprot_seqs.append(seq_found)
    time.sleep(1)

df['UniProt_AA_Sequences'] = uniprot_seqs
print("UniProt sequences (AA only) saved.")

UniProt sequences (AA only) saved.


In [33]:
#From this point I ran into issues trying to get the GPRs from NCBI Blast due to too many requests.
#What you'll need to do is download a local BLAST database (swissprot is relatively small). My follow-up script "BLAST_into_MATLAB" will continue this.
print(df)
###If there are any issues or if you have questions, contact me at dtillery@sdsu.edu

     Abbreviation                                        Description  \
0    2OH3K5MPPISO  2-hydroxy-3-keto-5-methylthiopentenyl-1-phosph...   
1         3HAD160  3-hydroxyacyl-[acyl-carrier-protein] dehydrata...   
2            ACLS                              Acetolactate synthase   
3          ADEtex  adenine transport via diffusion (extracellular...   
4           ALDD1             NAD+-linked formaldehyde dehydrogenase   
..            ...                                                ...   
908      DM_oaa_c                                oxaloacetate demand   
909      DM_akg_c                              2-oxoglutarate demand   
910     DM_3mob_c                     3-methyl-2-oxobutanoate demand   
911       SK_h2_c                                            H2 sink   
912      SK_for_c                                       formate sink   

                                              Reaction  \
0    2h3kmtp[c] + h2o[c]  -> 12dmpo[c] + h[c] + pi[c]    
1            3hpalm